In [1]:
from pathlib import Path
from typing import Tuple

import pandas as pd

from box import (Item, build_box, Warehouse, 
                 estimate_cost, split_box, sum_prototypes)
from adapter import BoxOrder, string_box
from tinydb import TinyDB, Query
from tinydb.operations import delete

db = TinyDB('db.json')
product_table = db.table('products')
menus_table = db.table('menus')

product_cache = {product['item_code']: Item(**product) 
                 for product in product_table.all()}
menu_cache = {menu['name']: menu['products'] 
              for menu in menus_table.all()}

In [129]:
pricing = Path(Path.home(), 'desktop', 'SeasonalityAnalysis', 'MGPricing100119to100121.xls')
all_purchases = pd.read_excel(pricing)

df = pd.read_csv(Path(Path.home(), 'Desktop', 'DELIVERIES.csv'))

completed = df[df['Delivery Status'].isin(['Completed', 'Future'])]

failed = df[df['Delivery Status'] == 'Failed']

In [130]:
purchases = all_purchases.copy()[all_purchases['Date Received'] >= pd.Timestamp('2020-10-01')]
purchases['unit price'] = purchases['FoodBank Cost/Value'] / purchases['Qty']

maxes = purchases.groupby('Item Code')['unit price'].agg(max)
maxes.name = "max"

mins = purchases.groupby('Item Code')['unit price'].agg(min)
mins.name = "min"

means = purchases.groupby('Item Code')['unit price'].agg('mean')
means.name = "mean"

price_summary = pd.concat([maxes, mins, means], axis=1)

In [131]:
min_lookup = price_summary['min'].to_dict()

def mute_price(item:dict) -> dict:
    return {key: val for key, val in item.__dict__.items() if key != 'price'}

price_muted = {key: mute_price(val) for key, val in product_cache.items()}

min_price_cache = {key: Item(**val, price=min_lookup[key]) for key, val in price_muted.items()}

In [132]:
warehouse = Warehouse(
    date=pd.Timestamp.now(),
    window='AM',
    substitutions=[],
    menus=menu_cache,
    items=product_cache
)

cheap_warehouse = Warehouse(
    date=pd.Timestamp.now(),
    window='AM',
    substitutions=[],
    menus=menu_cache,
    items=min_price_cache
)

order = BoxOrder(
    menu_name='Standard B Family',
)

In [133]:
def string_box(stop:dict) -> str:
    """
    A hack...
    """
    return  stop['Box Type'] + ' ' + stop['Box Menu'] + ' ' + stop['Box Size']

prototypes = [menu_cache[menu] for menu in string_box(completed)]

big_box_proto = sum_prototypes(prototypes)

big_box = build_box(big_box_proto)(warehouse)
cheap_big_box = build_box(big_box_proto)(cheap_warehouse)

In [134]:
total_cost = estimate_cost(big_box)
cheap_cost = estimate_cost(cheap_big_box)

print(f'Ave cost: {total_cost}')
print(f'"Cheap" cost: {cheap_cost}')
for (rack, items), (cheap_rack, cheap_items) in zip(split_box('type', big_box).items(), 
                                                    split_box('type', cheap_big_box).items()):
    
    print(f'{rack}: mean price: {round(estimate_cost(items), 2)} cheap price: {round(estimate_cost(cheap_items), 2)}')
    print(f'\t potential savings: {round(estimate_cost(items) - estimate_cost(cheap_items), 2)}')

Ave cost: 477933.8
"Cheap" cost: 393629.6496999999
07-Dairy: Yogurt, Cheese, Milk, Butter, Sour Cream, Ice Cream: mean price: 114901.61 cheap price: 93122.93
	 potential savings: 21778.68
15-Meat/Fish/Poultry: mean price: 88594.18 cheap price: 76442.74
	 potential savings: 12151.44
27-Vegetables - Canned & Frozen: mean price: 26942.75 cheap price: 26229.72
	 potential savings: 713.03
26-Spice/Condiments/Sauce: Herbs, Salt, Sugar, Mixes, Vinegar, Extracts, Mustard, Syrup, Jelly, Sauces, Salad Oil: mean price: 4776.03 cheap price: 5957.37
	 potential savings: -1181.34
10-Fruit: Canned & Frozen: mean price: 42168.54 cheap price: 41261.56
	 potential savings: 906.98
05-Cereal: Hot & Cold: mean price: 22558.28 cheap price: 21279.53
	 potential savings: 1278.75
23-Protein -Non-Meat: Peanut Butter, Beans, Eggs, Pork & Beans, Nuts: mean price: 23206.54 cheap price: 21517.85
	 potential savings: 1688.69
21-Pasta: Macaroni, Spaghetti, Noodles: mean price: 11308.63 cheap price: 12100.57
	 potenti

In [136]:
45100 / len(completed)

1.8547458463563087